# Create QKeras model

Import NN libraries, including QKeras

In [ ]:
from tensorflow.random import set_seed
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import History 
import matplotlib.pyplot as plt
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu

#Custom defined functions for data processing
from utils.utils import *

Import libraries for pruning:

In [ ]:
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning

Set random seeds:

In [ ]:
seed = 12345
np.random.seed(seed)
set_seed(seed)

Start by loading the dataset, and perform train/test split:

In [ ]:
X,Y = preproc()

x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2,random_state=seed)
initial_lr = 0.001
optimizer = Adam(learning_rate=initial_lr)

Define and train the quantized model (with pruning). Quantization is handled by **QKeras layers**, which requires (among other typical Keras options) the numerical precision: in this case _fixed point_ numbers (16,1), where 16 is the width and 1 is the integer part

In [ ]:
# create model
name="RMSE validation"
name2="RMSE training"

history = History()
model = Sequential()

model.add(QDense(60,  input_shape=(27,),kernel_quantizer=quantized_bits(16,1),bias_quantizer=quantized_bits(16,1), kernel_initializer='random_normal'))
model.add(QActivation(activation=quantized_relu(16,1), name='relu1'))
model.add(QDense(50,kernel_quantizer=quantized_bits(16,1),bias_quantizer=quantized_bits(16,1)))
model.add(QActivation(activation=quantized_relu(16,1), name='relu2'))
model.add(QDense(30,kernel_quantizer=quantized_bits(16,1),bias_quantizer=quantized_bits(16,1)))
model.add(QActivation(activation=quantized_relu(16,1), name='relu3'))
model.add(QDense(40,kernel_quantizer=quantized_bits(16,1),bias_quantizer=quantized_bits(16,1)))
model.add(QActivation(activation=quantized_relu(16,1), name='relu4'))
model.add(QDense(15,kernel_quantizer=quantized_bits(16,1),bias_quantizer=quantized_bits(16,1)))
model.add(QActivation(activation=quantized_relu(16,1), name='relu5'))
model.add(QDense(1,kernel_quantizer=quantized_bits(16,1)))
model.add(QActivation(activation=quantized_relu(16,1), name='relu6'))
    
## Define pruning schedule
pruning_params = {"pruning_schedule" : pruning_schedule.ConstantSparsity(0.75, begin_step=2000, frequency=100)} ## Add pruning wrapper around each model layer
model = prune.prune_low_magnitude(model, **pruning_params)
        
model.compile(loss='mean_squared_error', optimizer=optimizer)
model.fit(x_train, y_train,
        batch_size=256,
        epochs=10,
        verbose=1,
        validation_data=(x_valid, y_valid),
        callbacks=[history,pruning_callbacks.UpdatePruningStep()])
        
model = strip_pruning(model) # Remove the wrapping and remain with the "Keras-like" model
w = model.layers[0].weights[0].numpy()
h, b = np.histogram(w, bins=100)
plt.figure(figsize=(7,7))
plt.bar(b[:-1], h, width=b[1]-b[0])
plt.semilogy()
plt.show()

print('% of zeros = {}'.format(np.sum(w==0)/np.size(w)))

Perform model predictions, and save the model in a separate .h5 file

In [ ]:
train_predictions = model.predict(x_train)
predictions = model.predict(x_valid)
lin_mse = mean_squared_error(y_valid, predictions)
lin_rmse = np.sqrt(lin_mse)
lin_mse2 = mean_squared_error(y_train, train_predictions)
lin_rmse2 = np.sqrt(lin_mse2)
msg = "%s: %f" % (name, lin_rmse)
msg2 = "%s: %f" % (name2, lin_rmse2)
print(msg)
print(msg2)

model.save('model.h5')

Plot the regression scatter plot: measured p_T vs predicted p_T

In [ ]:
fig,ax = plt.subplots()
ax.scatter(y_valid, predictions, edgecolors=(0, 0, 0))
ax.set_title('Regression model predictions (validation set)')
ax.set_xlabel('Measured $p_T$ (GeV/c)')
ax.set_ylabel('Predicted $p_T$ (GeV/c)')
ax.plot([Y.min(), Y.max()], [Y.min(), Y.max()], 'k--', lw=4)
plt.rc('font', size=20)
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=18)    
plt.rc('xtick', labelsize=18)   
plt.rc('ytick', labelsize=18)  
plt.rc('legend', fontsize=18)    
plt.rc('figure', titlesize=18)
plt.tight_layout()
plt.show()

Plot the loss per epoch history, for both training and validation:

In [ ]:
fig2,ax2 = plt.subplots()
ax2.plot(history.history['loss'], label='loss')
ax2.plot(history.history['val_loss'], label='val_loss')
ax2.set_title('Training and Validation loss per epoch')
ax2.set_xlabel('# Epoch')
ax2.set_ylabel('loss')
plt.legend()
plt.tight_layout()
plt.show()